In [1]:
%cd /Users/hilarifan/Downloads

/Users/hilarifan/Downloads


In [2]:
import pandas as pd

data = pd.read_csv('data.csv') 
TWEETS_DIR = list(data['tweet'])
LABELS_DIR = list(data['valence'])
EMBEDDINGS_DIR = './glove.6B.50d.txt'

In [3]:
import numpy as np
TWEETS_DIR = np.array(TWEETS_DIR)
LABELS_DIR = np.array(LABELS_DIR)

In [4]:
#TODO: Shuffle data 
from sklearn.utils import shuffle

PERCENT_DATA_USED = .1
X_train, y_train = shuffle(TWEETS_DIR, LABELS_DIR)

#TODO: Data preprocessing: only use PERCENT_DATA_USED of the train/test images and labels
num_training = int(len(TWEETS_DIR) * PERCENT_DATA_USED)

X_train = X_train[:num_training]
y_train = y_train[:num_training]

In [5]:
%run -i load_tweet_data.py

tweets, tweets_preprocessed, labels, word_index, embedding_matrix = load_tweet_data(X_train, y_train, EMBEDDINGS_DIR)

1 -- Loading tweets and labels
2 -- Tokenizing the tweets: converting sentences to sequence of words
3 -- Padding sequences to ensure samples are the same size
4 -- Loading pre-trained word embeddings. This may take a few minutes.
5 -- Finding word embeddings for words in our tweets.


In [6]:
# What is the size of our dataset?

print("Training Data Size: ", tweets_preprocessed.shape)
print("Number of Tweets: ", tweets_preprocessed.shape[0])
print("Max Tweet Length: ", tweets_preprocessed.shape[1])
print()
print("Labels Size: ", labels.shape)

Training Data Size:  (160000, 45)
Number of Tweets:  160000
Max Tweet Length:  45

Labels Size:  (160000,)


In [7]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import metrics
from keras.models import Model
import pickle

In [8]:
model = Sequential()

# Add pre-trained embedding layer 
# converts word indices to GloVe word embedding vectors as they're fed in
model.add(Embedding(len(word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=tweets_preprocessed.shape[1],
                    trainable=False))

# At this point, each individual training sample is now a sequence of word embedding vectors

######################################
# TODO: define the rest of the network!

# First LSTM layer (return sequence so that we can feed the output into the 2nd LSTM layer)
model.add(LSTM(64, return_sequences = True, activation='relu'))
model.add(Dropout(.2))

# Second LSTM layer 
# Don't return sequence this time, because we're feeding into a fully-connected layer
model.add(LSTM(64, activation='relu'))
model.add(Dropout(.2))

# Dense 1
model.add(Dense(32, activation='relu'))
model.add(Dropout(.2))

# Dense 2 (final vote)
model.add(Dense(1, activation = 'sigmoid'))

######################################

print(model.summary())

LOSS = 'binary_crossentropy' 
OPTIMIZER = 'RMSprop' # RMSprop tends to work well for recurrent models

model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

2022-01-09 22:42:16.153706: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 45, 50)            6929350   
                                                                 
 lstm (LSTM)                 (None, 45, 64)            29440     
                                                                 
 dropout (Dropout)           (None, 45, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0

In [9]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(tweets_preprocessed, labels, train_size=0.8, test_size=0.2, shuffle=True, random_state=42)

In [10]:
# Train model
TEST_SIZE = 0.5

#####################################
# TODO: pick number of epochs and batch size

EPOCHS = 10
BATCH_SIZE = 128

#####################################

model.fit(X_train1, y_train1, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split = TEST_SIZE)

Epoch 1/10
500/500 [==============================] - 124s 237ms/step - loss: 0.6049 - binary_accuracy: 0.6744 - val_loss: 0.5647 - val_binary_accuracy: 0.7089
Epoch 2/10
500/500 [==============================] - 121s 242ms/step - loss: 0.5513 - binary_accuracy: 0.7217 - val_loss: 0.5283 - val_binary_accuracy: 0.7364
Epoch 3/10
500/500 [==============================] - 101s 203ms/step - loss: 0.5278 - binary_accuracy: 0.7373 - val_loss: 0.5232 - val_binary_accuracy: 0.7385
Epoch 4/10
500/500 [==============================] - 118s 236ms/step - loss: 0.5119 - binary_accuracy: 0.7476 - val_loss: 0.5228 - val_binary_accuracy: 0.7365
Epoch 5/10
500/500 [==============================] - 85s 171ms/step - loss: 0.5016 - binary_accuracy: 0.7548 - val_loss: 0.4943 - val_binary_accuracy: 0.7586
Epoch 6/10
500/500 [==============================] - 77s 154ms/step - loss: 0.4913 - binary_accuracy: 0.7637 - val_loss: 0.4905 - val_binary_accuracy: 0.7633
Epoch 7/10
500/500 [======================

In [11]:
#TODO: Evaluate model

test_loss, test_acc = model.evaluate(X_test1,  y_test1, verbose=2)
print("Test loss: ", test_loss, "\nTest accuracy: ", test_acc)

1000/1000 - 14s - loss: 0.4904 - binary_accuracy: 0.7644 - 14s/epoch - 14ms/step
Test loss:  0.49042174220085144 
Test accuracy:  0.7643749713897705
